In [1]:
# Importing Qiskit
from qiskit import *
# from qiskit.tools.visualization import plot_bloch_multivector
import random
from random import randrange

In [21]:
instruction = 'x q[0];'

# instruction.strip()
print(instruction.startswith('x'))
instruction = instruction.replace('0', '593')
print(instruction)

True
x q[593];


In [24]:
old_qr = int(instruction.split()[1][2:-2])
old_qr

593

In [ ]:
parsed_instructions = []
    for i, qasm_code in enumerate(qs):
        for line in qasm_code:
            line = line.strip()    # removing leading/trailing whitespace
            if line.startswith(('x', 'h', 'measure')):
                line = line.replace('0', i)
                parsed_instructions.append(line)              # Bug here : need to store the index as well


In [3]:
def NoisyChannel(qc1, qc2, qc1_name, rate = 0.05):
    ''' This function takes the output of a circuit qc1 (made up only of x and 
        h gates), simulates a noisy quantum channel where Pauli errors (X - bit flip; Z - phase flip)
        will occur in qc2, and then initializes another circuit qc2 with the introduced noise.
    ''' 
    
    # Retrieve quantum state from qasm code of qc1
    qs = [qc1[i].qasm().split('\n') for i in range(len(qc1))]
    
    # Debugging: Print the QASM instructions
    # print("QASM Instructions:")
    # for qasm_code in qs:
    #     for line in qasm_code:
    #         print(line)
    
    # Process the code to get the instructions
    parsed_instructions = []
    for qasm_code in qs:
        for line in qasm_code:
            line = line.strip()
            if line.startswith(('x', 'h', 'measure')):
                parsed_instructions.append(line)

    # Debugging: Print the parsed instructions
    # print("Parsed Instructions:", parsed_instructions)
    
    # Apply parsed instructions to qc2
    for instruction in parsed_instructions:
        if instruction.startswith('x'):
            old_qr = int(instruction.split()[1][2])
            qc2[old_qr].x(0)
            
        elif instruction.startswith('h'):
            old_qr = int(instruction.split()[1][2])
            qc2[old_qr].h(0)
        
        elif instruction.startswith('measure'):
            continue  # exclude measuring
            
        else:
            print(f"Unable to parse instruction: {instruction}")
            raise Exception('Unable to parse instruction')
    
    # Introducing noise (1/7 probability)
    for instruction in parsed_instructions:
        if random.random() < rate:
            old_qr = int(instruction.split()[1][2])
            qc2[old_qr].x(0)  # Apply bit-flip error
            
        if random.random() < rate:
            old_qr = int(instruction.split()[1][2])
            qc2[old_qr].z(0)  # Apply phase-flip error

In [4]:
def generate_random_bases(num_of_bases):
    """This function selects a random basis for each bit"""
    
    bases_string = ""
    for i in range(num_of_bases):
        randBasis = random.randint(0, 1) # Flip Coin

        if randBasis == 0:
            bases_string += "Z" 
        else:
            bases_string += "X"
            
    return bases_string

In [5]:
def encode(bits, bases):
    """This function encodes each bit into the given basis."""
    
    encoded_qubits = []
    
    for bit, basis in zip(bits, bases):
        qc = QuantumCircuit(1, 1) # Create a quantum circuit for each qubit
        
        # Possible Cases
        if bit=="0" and basis == "Z":
            encoded_qubits.append(qc) # Do not apply any gates

        elif bit=="1" and basis == "Z":
            qc.x(0) # Apply X Gate
            encoded_qubits.append(qc)

        elif bit=="0" and basis == "X":
            qc.h(0) # Apply H Gate
            encoded_qubits.append(qc)

        elif bit=="1" and basis == "X":
            qc.x(0) # Apply X Gate
            qc.h(0) # Apply H Gate
            encoded_qubits.append(qc)
            
    return (encoded_qubits)

In [6]:
def measure(qubits, bases):
    """This function measures each qubit in the corresponding basis chosen for it."""

    bits = "" # The results of measurements

    for qubit, basis in zip(qubits, bases):

        # Add measurement depending on basis
        if basis == "Z":
            qubit.measure(0, 0)
        elif basis == "X":
            qubit.h(0)
            qubit.measure(0, 0)

        # Execute on Simulator
        simulator = Aer.get_backend('qasm_simulator')
        result = execute(qubit, backend=simulator, shots=1).result()
        counts = result.get_counts()
        measured_bit = max(counts, key=counts.get) # Max doesn't matter for simulator since there is only one shot.

        bits += measured_bit
        
    return bits

# Start

In [7]:
# Preparation for encoding
KEY_LENGTH = n
random.seed(0) # Seed the random number generator. This will be used as our "coin flipper".

# Alice

In [8]:
#alice
# alice = QuantumCircuit(qreg, creg, name='alice') ##

# Generating a random string of bits
alice_bits = ""
for i in range(KEY_LENGTH):
    randBit = random.randint(0, 1) # Flip Coin
    alice_bits += str(randBit) # Add randomly chosen bit to the bit string.
    
print("The bits Alice is going to send are: " + alice_bits[:10] + "...")

The bits Alice is going to send are: 1101111110...


In [9]:
alice_bases = generate_random_bases(KEY_LENGTH) # Alice randomly chooses a basis for each bit.
    
print("The bases Alice is going to encode them in are: " + alice_bases[:10] + "...")

The bases Alice is going to encode them in are: ZXXZZZZXZX...


## Encoding

In [10]:
# Encode Alice's bits
encoded_qubits = encode(alice_bits, alice_bases)

# Print circuits for first 5 qubits.
for i in range(5):
    print(encoded_qubits[i])

print("etc.")

     ┌───┐
  q: ┤ X ├
     └───┘
c: 1/═════
          
     ┌───┐┌───┐
  q: ┤ X ├┤ H ├
     └───┘└───┘
c: 1/══════════
               
     ┌───┐
  q: ┤ H ├
     └───┘
c: 1/═════
          
     ┌───┐
  q: ┤ X ├
     └───┘
c: 1/═════
          
     ┌───┐
  q: ┤ X ├
     └───┘
c: 1/═════
          
etc.


## Quantum Signal Channel

In [11]:
qubits_received = [QuantumCircuit(1, 1) for _ in range(len(encoded_qubits))]
NoisyChannel(encoded_qubits, qubits_received, 'alice') ##alice sends noisy states to Bob

#QUANTUM_CHANNEL = encoded_qubits

# Bob

In [12]:
bob_bases = generate_random_bases(KEY_LENGTH) # Bob randomly chooses a basis for each bit.
    
print("The bases Bob is going to decode them in are: " + bob_bases[:10] + "...")

The bases Bob is going to decode them in are: ZZXZZZZXXZ...


## Measurement

In [13]:
# qubits_received = QUANTUM_CHANNEL # Receive qubits from quantum channel
bob_bits = measure(qubits_received, bob_bases)

print("The first few bits Bob received are: " + bob_bits[:10] + "...")

The first few bits Bob received are: 1000000100...


# Public Interaction Channel

In [14]:
CLASSICAL_CHANNEL = alice_bases # Alice tells Bob which bases she used

In [15]:
# Store the indices of the bases they share in common
common_bases = [i for i in range(KEY_LENGTH) if CLASSICAL_CHANNEL[i] == bob_bases[i]]

print("The indices of the first 10 bases they share in common are: " + str(common_bases[:10]))

The indices of the first 10 bases they share in common are: [0, 2, 3, 4, 5, 6, 7, 12, 14, 15]


In [16]:
bob_bits = [bob_bits[index] for index in common_bases]

## Comparision

In [17]:
CLASSICAL_CHANNEL = common_bases # Bob tells Alice which bases they shared in common

In [18]:
alice_bits = [alice_bits[index] for index in common_bases] # Alice keeps only the bits they shared in common

## Spotting a few qubits

In [19]:
CLASSICAL_CHANNEL = alice_bits[:100] # Alice tells Bob the first 100 bits she has left.

# Bob checks if they match the first 100 bits that he has
if CLASSICAL_CHANNEL == bob_bits[:100]:
    print("Yep, Alice and Bob seem to have the same bits!")
else:
    print("Uh oh, at least one of the bits is different.")

Uh oh, at least one of the bits is different.


In [20]:
error_count = sum(1 for i in range(100) if CLASSICAL_CHANNEL[i] != bob_bits[i])
print(" The error is : ", error_count/100)

 The error count is :  45


## Final Key : No interception

In [21]:
alice_bits = alice_bits[100:] # Alice discards the first 100 bits
bob_bits = bob_bits[100:] # Alice discards the first 100 bits

In [22]:
key = "" 
for bit in alice_bits: # Or bob_bits, since both should be the same
    key += bit

print("The key is:")
print(str(key))

print("\nThe key is " + str(len(key)) + " bits long.")

The key is:
010001111000000100111001111100101011010010011101100100000101100010101000000010111011000100001010111110011011100101100111010011011010110000101010111111101100000

The key is 159 bits long.


# Interception

In [23]:
# Generating a random string of bits
alice_bits = ""
for i in range(KEY_LENGTH):
    randBit = random.randint(0, 1) # Flip Coin
    alice_bits += str(randBit) # Add randomly chosen bit to the bit string.
    
# Alice randomly chooses a basis for each bit.
alice_bases = generate_random_bases(KEY_LENGTH)

# Encode Alice's bits
encoded_qubits = encode(alice_bits, alice_bases)
qubits_intercepted = [QuantumCircuit(1, 1) for _ in range(len(encoded_qubits))]

# QUANTUM_CHANNEL = encoded_qubits
NoisyChannel(encoded_qubits, qubits_intercepted, 'alice') ##Eve intercepts noisy states 

# Eve

In [24]:
# qubits_intercepted = QUANTUM_CHANNEL # Intercept qubits
eve_bases = generate_random_bases(KEY_LENGTH) # Generate a random set of bases
eve_bits = measure(qubits_intercepted, eve_bases) # Measure the qubits

In [25]:
# Eve encodes her decoy qubits and sends them along the quantum channel
# QUANTUM_CHANNEL = encode(eve_bits, eve_bases)
encoded_intercepted_qubits = encode(eve_bits, eve_bases) ##
NoisyChannel(encoded_intercepted_qubits, qubits_received, 'eve') ##Eve sends intercepted noisy states to Bob 

In [26]:
bob_bases = generate_random_bases(KEY_LENGTH) # Bob randomly chooses a basis for each bit.
# qubits_received = QUANTUM_CHANNEL # Receive qubits from quantum channel
bob_bits = measure(qubits_received, bob_bases)

## Comparision

In [27]:
CLASSICAL_CHANNEL = alice_bases # Alice tells Bob which bases she used

In [28]:
# Store the indices of the bases they share in common
common_bases = [i for i in range(KEY_LENGTH) if CLASSICAL_CHANNEL[i]==bob_bases[i]]
bob_bits = [bob_bits[index] for index in common_bases]

In [29]:
CLASSICAL_CHANNEL = common_bases # Bob tells Alice which bases they shared in common
alice_bits = [alice_bits[index] for index in common_bases] # Alice keeps only the bits they shared in common

## Detection

In [30]:
CLASSICAL_CHANNEL = alice_bits[:100] # Alice tells Bob the first 100 bits she has left.

# Bob checks if they match the first 100 bits that he has
if CLASSICAL_CHANNEL == bob_bits[:100]:
    print("Yep, Alice and Bob seem to have the same bits!")
else:
    print("Uh oh, at least one of the bits is different.")

Uh oh, at least one of the bits is different.


# QBER

In [31]:
rounds = len(alice_bits)//3
errors = 0

for i in range(rounds):
    bit_index = random.randrange(len(alice_bits)) 
    # tested_bit = alice_key[bit_index]
    
    if alice_bits[bit_index] != bob_bits[bit_index]: #comparing tested rounds
        errors += 1 #calculating errors
        
    del alice_bits[bit_index] #removing tested bits from key strings
    del bob_bits[bit_index]

QBER = errors/rounds #calculating QBER
QBER = round(QBER,2) #saving the answer to two decimal places

print("QBER value =", QBER)
print("alices secret key =", alice_bits)
print("Bob secret key =", bob_bits)

QBER value = 0.5
alices secret key = ['1', '0', '0', '1', '0', '1', '1', '1', '0', '0', '1', '1', '1', '0', '1', '1', '1', '0', '1', '0', '1', '1', '0', '0', '0', '1', '1', '0', '1', '1', '1', '1', '0', '0', '1', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '1', '0', '1', '0', '1', '1', '1', '1', '0', '0', '1', '0', '1', '1', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '1', '0', '1', '0', '0', '1', '1', '1', '1', '1', '1', '0', '1', '1', '0', '0', '1', '0', '0', '0', '1', '1', '1', '0', '0', '0', '1', '1', '1', '1', '1', '0', '1', '0', '0', '0', '0', '0', '1', '1', '0', '0', '1', '0', '0', '0', '1', '1', '0', '1', '0', '0', '0', '0', '1', '1', '1', '0', '0', '0', '0', '1', '0', '1', '0', '0', '1', '1', '1', '1', '0', '1', '1', '0', '1', '1', '0', '1', '1', '1', '0', '0', '1', '1', '1', '1', '0', '0', '1', '0', '1', '1', '1', '1', '0', '1', '1', '0', '0', '0']
Bob secret key = ['1', '1', '0', '0', '1', '0', '1', '1', '0', '1', '0', '1', '0', '0', '0', '1', '0', '0', '0',

In [32]:
errors, rounds

(42, 84)